What are the regional patterns of health graduate employment by gender in Ireland, and which HSE regions show the greatest gender disparities? 

Focus: Compare male vs. female distribution across 6 HSE regions for nursing, medicine, and social care graduates. 

In [1]:
# %pip install plotly
# %pip install --upgrade nbformat

# Imports
import pandas as pd
import numpy as np

In [2]:
# Load in the CSV for graduate regions by gender 
file_path = 'csv_version/graduate_regions_by_gender.csv'
df = pd.read_csv(file_path)
df.head()

STATISTIC             Statistic Label  TLIST(A1)  Graduation Year  \
0  HGO22C01  Number of Health Graduates       2010             2010   
1  HGO22C01  Number of Health Graduates       2010             2010   
2  HGO22C01  Number of Health Graduates       2010             2010   
3  HGO22C01  Number of Health Graduates       2010             2010   
4  HGO22C01  Number of Health Graduates       2010             2010   

   C04477V05260         Field of Study C03919V04671 Gender C04300V05079  \
0            30  Nursing and Midwifery           10   Male            -   
1            30  Nursing and Midwifery           10   Male           20   
2            30  Nursing and Midwifery           10   Male           10   
3            30  Nursing and Midwifery           10   Male           30   
4            30  Nursing and Midwifery           10   Male           50   

          HSE Health Regions    UNIT  VALUE  
0            All HSE Regions  Number     54  
1    HSE Dublin and Midlands  Number      9  
2  HSE Dublin and North East  Number     12  
3  HSE Dublin and South East  Number      9  
4                HSE Midwest  Number      3

In [3]:
# Intial inspection of the data
print("shape of dataset", df.shape)
print("column names:")
print(df.columns.tolist())

shape of dataset (756, 12)
column names:
['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'Graduation Year', 'C04477V05260', 'Field of Study', 'C03919V04671', 'Gender', 'C04300V05079', 'HSE Health Regions', 'UNIT', 'VALUE']


In [4]:
# is there missing rows?
print(df.isna().sum())

STATISTIC             0
Statistic Label       0
TLIST(A1)             0
Graduation Year       0
C04477V05260          0
Field of Study        0
C03919V04671          0
Gender                0
C04300V05079          0
HSE Health Regions    0
UNIT                  0
VALUE                 0
dtype: int64


No values are missing but in the head there is a dash for - C03919V04671 in the this is could be missing data I need the descriptor for this DS

https://db.nomics.world/CSO/HEO04/HEO04.-.-.-.IEZ999?tab=dimensions
cites that C03919V04671 references gender, 10 for male, 20 for female, and - for both genders 

In [5]:
print("Unique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} → {df[col].unique()[:10]}")

Unique values per column:
STATISTIC: 1 → ['HGO22C01']
Statistic Label: 1 → ['Number of Health Graduates']
TLIST(A1): 12 → [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
Graduation Year: 12 → [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
C04477V05260: 3 → [ 30 202 702]
Field of Study: 3 → ['Nursing and Midwifery' 'Medicine' 'Social Care']
C03919V04671: 3 → ['10' '20' '-']
Gender: 3 → ['Male' 'Female' 'All genders']
C04300V05079: 7 → ['-' '20' '10' '30' '50' '60' '40']
HSE Health Regions: 7 → ['All HSE Regions' 'HSE Dublin and Midlands' 'HSE Dublin and North East'
 'HSE Dublin and South East' 'HSE Midwest' 'HSE West and North West'
 'HSE South West']
UNIT: 1 → ['Number']
VALUE: 132 → [ 54   9  12   3 786 153 150 120  78 132]


C04300V05079 may appears to be the HSE region this row can likely be dropped

Rows to drop as there are value not needed 

| Column             | Keep or drop |
|--------------------|-------------|
| Statistic          | drop        |
|Statistic Label:    | drop        |
| TLIST              | drop        |
| Graduation Year    | keep        |
| C04477V05260       | drop        |
| Field of Study     | keep        |
| C03919V04671       | drop        |
| Gender             | keep        |
| C04300V05079       | drop        |
| HSE Health Regions | keep        |
| UNIT               | drop        |
| VALUE              | keep        |


TLIST -> is the same as graduation year
C04477V05260 -> is tied to field of study
C03919V04671 -> is tied to gender
C04300V05079 -> is HSE region
Unit -> is the one value we do not need it overall  
Statistic is also the one value tied to the dataset   


It would also be interesting to break off a data frame of just all regions I think


In [6]:
# list of columns to drop
cols_to_drop = ['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'C04477V05260','C03919V04671', 'C04300V05079', 'UNIT']

# create a clean data frame
df_clean = df.drop(columns = cols_to_drop)
df_clean.head()

Graduation Year         Field of Study Gender         HSE Health Regions  \
0             2010  Nursing and Midwifery   Male            All HSE Regions   
1             2010  Nursing and Midwifery   Male    HSE Dublin and Midlands   
2             2010  Nursing and Midwifery   Male  HSE Dublin and North East   
3             2010  Nursing and Midwifery   Male  HSE Dublin and South East   
4             2010  Nursing and Midwifery   Male                HSE Midwest   

   VALUE  
0     54  
1      9  
2     12  
3      9  
4      3

In [7]:
# remove all HSE regions rows and all genders rows
# these will be brought in later as their own dataframe for exploration later they are already presnt in the data so we don't need it as it will skew results
df_clean = df_clean[
    (df_clean['Gender'] != 'All genders') &
    (df_clean['HSE Health Regions'] != 'All HSE Regions')
].copy()

df_clean.head()


Graduation Year         Field of Study Gender         HSE Health Regions  \
1             2010  Nursing and Midwifery   Male    HSE Dublin and Midlands   
2             2010  Nursing and Midwifery   Male  HSE Dublin and North East   
3             2010  Nursing and Midwifery   Male  HSE Dublin and South East   
4             2010  Nursing and Midwifery   Male                HSE Midwest   
5             2010  Nursing and Midwifery   Male    HSE West and North West   

   VALUE  
1      9  
2     12  
3      9  
4      3  
5      9

In [8]:
# shape of the clean df
df_clean.shape

(432, 5)

In [9]:
# summary of the clean dataset

dataset_summary = {
    'Rows': len(df_clean),
    'Years': f"{df_clean['Graduation Year'].min()}–{df_clean['Graduation Year'].max()}",
    'Fields of Study': df_clean['Field of Study'].nunique(),
    'Genders': df_clean['Gender'].unique().tolist(),
    'Regions': df_clean['HSE Health Regions'].nunique(),
    'Total Graduates': df_clean['VALUE'].sum()
}

print(dataset_summary)
# this summary needs to be tidied up in a readable format later 

{'Rows': 432, 'Years': '2010–2021', 'Fields of Study': 3, 'Genders': ['Male', 'Female'], 'Regions': 6, 'Total Graduates': np.int64(22626)}


In [10]:
# Make the all regions df, all genders 

df_all_regions = df[
    (df['HSE Health Regions'] == 'All HSE Regions') &
    (df['Gender'] != 'All genders')
].copy()


df_all_genders = df[
    (df['Gender'] == 'All genders') &
    (df['HSE Health Regions'] != 'All HSE Regions')
].copy()


In [11]:
df_all_regions.head()

STATISTIC             Statistic Label  TLIST(A1)  Graduation Year  \
0   HGO22C01  Number of Health Graduates       2010             2010   
7   HGO22C01  Number of Health Graduates       2010             2010   
21  HGO22C01  Number of Health Graduates       2010             2010   
28  HGO22C01  Number of Health Graduates       2010             2010   
42  HGO22C01  Number of Health Graduates       2010             2010   

    C04477V05260         Field of Study C03919V04671  Gender C04300V05079  \
0             30  Nursing and Midwifery           10    Male            -   
7             30  Nursing and Midwifery           20  Female            -   
21           202               Medicine           10    Male            -   
28           202               Medicine           20  Female            -   
42           702            Social Care           10    Male            -   

   HSE Health Regions    UNIT  VALUE  
0     All HSE Regions  Number     54  
7     All HSE Regions  Number    786  
21    All HSE Regions  Number     51  
28    All HSE Regions  Number     96  
42    All HSE Regions  Number     48

In [12]:
df_all_genders.head()

STATISTIC             Statistic Label  TLIST(A1)  Graduation Year  \
15  HGO22C01  Number of Health Graduates       2010             2010   
16  HGO22C01  Number of Health Graduates       2010             2010   
17  HGO22C01  Number of Health Graduates       2010             2010   
18  HGO22C01  Number of Health Graduates       2010             2010   
19  HGO22C01  Number of Health Graduates       2010             2010   

    C04477V05260         Field of Study C03919V04671       Gender  \
15            30  Nursing and Midwifery            -  All genders   
16            30  Nursing and Midwifery            -  All genders   
17            30  Nursing and Midwifery            -  All genders   
18            30  Nursing and Midwifery            -  All genders   
19            30  Nursing and Midwifery            -  All genders   

   C04300V05079         HSE Health Regions    UNIT  VALUE  
15           20    HSE Dublin and Midlands  Number    162  
16           10  HSE Dublin and North East  Number    162  
17           30  HSE Dublin and South East  Number    129  
18           50                HSE Midwest  Number     81  
19           60    HSE West and North West  Number    162

In [13]:
# all genders and all regions would be nice too so this is below
df_fully_aggregated = df[
    (df['Gender'] == 'All genders') &
    (df['HSE Health Regions'] == 'All HSE Regions')
].copy()
df_fully_aggregated

STATISTIC             Statistic Label  TLIST(A1)  Graduation Year  \
14   HGO22C01  Number of Health Graduates       2010             2010   
35   HGO22C01  Number of Health Graduates       2010             2010   
56   HGO22C01  Number of Health Graduates       2010             2010   
77   HGO22C01  Number of Health Graduates       2011             2011   
98   HGO22C01  Number of Health Graduates       2011             2011   
119  HGO22C01  Number of Health Graduates       2011             2011   
140  HGO22C01  Number of Health Graduates       2012             2012   
161  HGO22C01  Number of Health Graduates       2012             2012   
182  HGO22C01  Number of Health Graduates       2012             2012   
203  HGO22C01  Number of Health Graduates       2013             2013   
224  HGO22C01  Number of Health Graduates       2013             2013   
245  HGO22C01  Number of Health Graduates       2013             2013   
266  HGO22C01  Number of Health Graduates       2014             2014   
287  HGO22C01  Number of Health Graduates       2014             2014   
308  HGO22C01  Number of Health Graduates       2014             2014   
329  HGO22C01  Number of Health Graduates       2015             2015   
350  HGO22C01  Number of Health Graduates       2015             2015   
371  HGO22C01  Number of Health Graduates       2015             2015   
392  HGO22C01  Number of Health Graduates       2016             2016   
413  HGO22C01  Number of Health Graduates       2016             2016   
434  HGO22C01  Number of Health Graduates       2016             2016   
455  HGO22C01  Number of Health Graduates       2017             2017   
476  HGO22C01  Number of Health Graduates       2017             2017   
497  HGO22C01  Number of Health Graduates       2017             2017   
518  HGO22C01  Number of Health Graduates       2018             2018   
539  HGO22C01  Number of Health Graduates       2018             2018   
560  HGO22C01  Number of Health Graduates       2018             2018   
581  HGO22C01  Number of Health Graduates       2019             2019   
602  HGO22C01  Number of Health Graduates       2019             2019   
623  HGO22C01  Number of Health Graduates       2019             2019   
644  HGO22C01  Number of Health Graduates       2020             2020   
665  HGO22C01  Number of Health Graduates       2020             2020   
686  HGO22C01  Number of Health Graduates       2020             2020   
707  HGO22C01  Number of Health Graduates       2021             2021   
728  HGO22C01  Number of Health Graduates       2021             2021   
749  HGO22C01  Number of Health Graduates       2021             2021   

     C04477V05260         Field of Study C03919V04671       Gender  \
14             30  Nursing and Midwifery            -  All genders   
35            202               Medicine            -  All genders   
56            702            Social Care            -  All genders   
77             30  Nursing and Midwifery            -  All genders   
98            202               Medicine            -  All genders   
119           702            Social Care            -  All genders   
140            30  Nursing and Midwifery            -  All genders   
161           202               Medicine            -  All genders   
182           702            Social Care            -  All genders   
203            30  Nursing and Midwifery            -  All genders   
224           202               Medicine            -  All genders   
245           702            Social Care            -  All genders   
266            30  Nursing and Midwifery            -  All genders   
287           202               Medicine            -  All genders   
308           702            Social Care            -  All genders   
329            30  Nursing and Midwifery            -  All genders   
350           202               Medicine            -  All genders   
371           702            Social Care  

In [14]:
# make a pivot  for gender divide
pivot_df = df_clean.pivot_table(
    index=['Graduation Year', 'Field of Study', 'HSE Health Regions'],
    columns='Gender',
    values='VALUE',
    aggfunc='sum',
    fill_value=0
).reset_index()

pivot_df['Total Graduates'] = pivot_df['Female'] + pivot_df['Male']
pivot_df['Female %'] = (pivot_df['Female'] / pivot_df['Total Graduates']) * 100
pivot_df['Male %'] = (pivot_df['Male'] / pivot_df['Total Graduates']) * 100
pivot_df['Gender Gap (%)'] = pivot_df['Female %'] - pivot_df['Male %']
pivot_df['Gender Ratio (F/M)'] = pivot_df['Female'] / pivot_df['Male']


In [15]:
pivot_df.head(10)

Gender  Graduation Year         Field of Study         HSE Health Regions  \
0                  2010               Medicine    HSE Dublin and Midlands   
1                  2010               Medicine  HSE Dublin and North East   
2                  2010               Medicine  HSE Dublin and South East   
3                  2010               Medicine                HSE Midwest   
4                  2010               Medicine             HSE South West   
5                  2010               Medicine    HSE West and North West   
6                  2010  Nursing and Midwifery    HSE Dublin and Midlands   
7                  2010  Nursing and Midwifery  HSE Dublin and North East   
8                  2010  Nursing and Midwifery  HSE Dublin and South East   
9                  2010  Nursing and Midwifery                HSE Midwest   

Gender  Female  Male  Total Graduates   Female %     Male %  Gender Gap (%)  \
0           18     6               24  75.000000  25.000000       50.000000   
1           27    12               39  69.230769  30.769231       38.461538   
2           21    18               39  53.846154  46.153846        7.692308   
3            3     3                6  50.000000  50.000000        0.000000   
4           15     6               21  71.428571  28.571429       42.857143   
5            9     6               15  60.000000  40.000000       20.000000   
6          153     9              162  94.444444   5.555556       88.888889   
7          150    12              162  92.592593   7.407407       85.185185   
8          120     9              129  93.023256   6.976744       86.046512   
9           78     3               81  96.296296   3.703704       92.592593   

Gender  Gender Ratio (F/M)  
0                 3.000000  
1                 2.250000  
2                 1.166667  
3                 1.000000  
4                 2.500000  
5                 1.500000  
6                17.000000  
7                12.500000  
8                13.333333  
9                26.000000

In [17]:
# export the pivot
pivot_df.to_csv("cleaned_gender_pivot.csv", index=False)